In [6]:
import math

def maxdeflection(clearspan, membertype):
    """
    NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.
    
    Parameters:
    clearspan: span length (mm)
    membertype:
        'flat-roofs'    = flat roof members due to maximum lr and r
        'floor'         = floor members due to l
        'roof-damaged'  = roof or floors that are likely to be damaged by large deflections
        'roof-undamaged'= roof or floors that are not likely to be damaged by large deflection
    Returns:
    Maximum permissible deflection (mm)
    """
    if membertype == 'flat-roofs':
        return clearspan / 180  # mm
    elif membertype == 'floor':
        return clearspan / 360  # mm
    elif membertype == 'roof-damaged':
        return clearspan / 480  # mm
    elif membertype == 'roof-undamaged':
        return clearspan / 240  # mm
    else:
        raise ValueError("Invalid member type. Must be flat-roofs, floor, roof-damaged, or roof-undamaged")

def momentofinertia(width, height):
    """
    Calculate gross moment of inertia.
    
    Parameters:
    width: width of the section (mm)
    height: height of the section (mm)
    Returns:
    Gross moment of inertia (mm^4)
    """
    ig = width * math.pow(height, 3) / 12  # mm^4
    return ig

def modulusofrupture(ig, distance, maximummoment):
    """
    Calculate modulus of rupture.
    
    Parameters:
    ig: gross moment of inertia (mm^4)
    distance: distance from neutral axis to extreme fiber (mm)
    maximummoment: maximum moment (N-mm)
    Returns:
    Modulus of rupture (N/mm^2)
    """
    sectionmodulus = ig / distance  # mm^3
    fr = maximummoment / sectionmodulus  # N/mm^2
    return fr

def crackingmoment(fr, ig, height):
    """
    NSCP 2015 Eq. 424.2.3.5b. Cracking moment.
    
    Parameters:
    fr: modulus of rupture (N/mm^2)
    ig: gross moment of inertia (mm^4)
    height: total height of the section (mm)
    Returns:
    Cracking moment (N-mm)
    """
    yt = height / 2  # mm
    mcr = (fr * ig) / yt  # N-mm
    return mcr

def calculate_modularratio(Es, Ec):
    """
    Calculate the modular ratio and round up to the nearest whole number.
    
    Parameters:
    Es: modulus of elasticity of steel (N/mm^2) 
    Ec: modulus of elasticity of concrete (N/mm^2)  
    Returns:
    Modular ratio (dimensionless, minimum 6)
    """
    ratio = math.ceil(Es / Ec)
    if ratio >= 6:
        modularratio = ratio
    else:
        modularratio = 6
    return modularratio

def crackedmomentofinertia(width, distance, modularratio, asbar, depth):
    """
    Calculate cracked moment of inertia.
    
    Parameters:
    width: width of the section (mm) 
    distance: distance from neutral axis to extreme fiber (mm)  
    modularratio: modular ratio (dimensionless)
    asbar: area of reinforcement (mm^2) 
    depth: overall depth of the section (mm) 
    Returns:
    Cracked moment of inertia (mm^4)
    """
    icr = width * math.pow(distance, 3) / 3 + modularratio * asbar * math.pow((depth - distance), 2)  # mm^4
    return icr

def effectivemomentofinertia(mcr, ma, ig, icr):
    """
    NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia.
    
    Parameters:
    mcr: cracking moment (N-mm)
    ma: applied moment (N-mm)
    ig: gross moment of inertia (mm^4)  
    icr: cracked section moment of inertia (mm^4)  
    Returns:
    Effective moment of inertia (mm^4)  # mm^4
    """
    ratio = math.pow(mcr / ma, 3)
    ie = ratio * ig + (1 - ratio) * icr  # mm^4
    if ie > ig:
        ie = ig  # mm^4
    return ie

def shortdeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload):
    """
    Calculates load (short-term) deflection based on beam type.
    
    Parameters:
    beamtype: beam configuration (str)
              'PR'  = pin roller (simply supported)
              'FFr' = fix free (cantilever)
              'FF'  = fix fix (fixed-fixed)
              'FR'  = fix roller (fixed-propped)
    clearspan: span length (mm) 
    elasticity: modulus of elasticity (N/mm^2)  
    ig: gross moment of inertia (mm^4)  
    pointload: point load (N)  
    uniformload: uniform load (N/mm)  
    Returns:
    Short-term deflection (mm) 
    """
    loaddeflectionvalue = 0
    if beamtype == 'PR':  # pin roller (simply supported)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (48 * elasticity * ig)  # mm
        if uniformload:
            loaddeflectionvalue += (5 * uniformload * math.pow(clearspan, 4)) / (384 * elasticity * ig)  # mm
    elif beamtype == 'FFr':  # fix free (cantilever)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (3 * elasticity * ig)  # mm
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (8 * elasticity * ig)  # mm
    elif beamtype == 'FF':  # fix fix (fixed-fixed)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (192 * elasticity * ig)  # mm
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (384 * elasticity * ig)  # mm
    elif beamtype == 'FR':  # fix roller (fixed-propped)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (48 * elasticity * ig)  # mm
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (185 * elasticity * ig)  # mm
    else:
        raise ValueError("Invalid beam type specified. Must be PR, FFr, FF, or FR.")
    return loaddeflectionvalue

def timedependentfactor(sustainedduration):
    """
    NSCP 2015 Table 424.2.4.1.3.
    Time-dependent factor for sustained loads.
    
    Parameters:
    sustainedduration: duration of sustained load in months (3, 6, 12, or >=60)  # months
    Returns:
    Time-dependent factor (dimensionless)
    """
    if sustainedduration == 3:
        timefactor = 1.0
    elif sustainedduration == 6:
        timefactor = 1.2
    elif sustainedduration == 12:
        timefactor = 1.4
    elif sustainedduration >= 60:
        timefactor = 2.0
    else:
        raise ValueError("Invalid sustainedduration. Must be 3, 6, 12, or >=60")
    return timefactor

def longdeflection(shortdeflectionvalue, timefactor, compressionbar, width, depth):
    """
    NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
    Additional deflection due to creep and shrinkage is the product of the immediate deflection and a factor lambda_delta.
    
    Parameters:
    shortdeflectionvalue: immediate deflection due to sustained load (mm) 
    timefactor: time-dependent factor (dimensionless)
    compressionbar: diameter of the compression bar (mm)  
    width: width of the section (mm) 
    depth: overall depth of the section (mm) 
    Returns:
    Long-term deflection (mm)  
    """
    asprime = math.pi * math.pow(compressionbar / 2, 2)  # mm^2
    rhoprime = asprime / (width * depth)  # dimensionless
    lambda_delta = timefactor / (1 + 50 * rhoprime)  # dimensionless
    timedependentdeflectionvalue = shortdeflectionvalue * lambda_delta  # mm
    return timedependentdeflectionvalue

def finaldeflection(maxdeflection, longdeflectionvalue, shortdeflectionvalue):
    """
    Determines the governing deflection as the sum of the time-dependent and load deflections,
    but capped by the maximum permissible deflection.
    
    Parameters:
    maxdeflection: maximum permissible deflection (mm)  # mm
    longdeflectionvalue: long-term deflection (mm)  # mm
    shortdeflectionvalue: short-term deflection (mm)  # mm
    Returns:
    Final deflection (mm)  # mm
    """
    totaldeflection = longdeflectionvalue + shortdeflectionvalue  # mm
    if totaldeflection <= maxdeflection:
        return totaldeflection  # mm
    else:
        return maxdeflection  # mm

if __name__ == '__main__':
    clearspan = 6000  # mm
    membertype = 'roof-undamaged'  # options: 'flat-roofs', 'floor', 'roof-damaged', 'roof-undamaged'
    width = 200   # mm
    height = 300  # mm
    distance = height / 2  # mm (distance from neutral axis to extreme fiber)
    maximummoment = 1e9  # N-mm
    fcprime = 21  # MPa
    Es = 200000  # N/mm^2
    Ec = 4700 * math.sqrt(fcprime)  # N/mm^2
    elasticity = 10000  # N/mm^2, modulus of elasticity of Steel-Concrete
    barsize = 16  # mm
    asbar = math.pi * math.pow(barsize / 2, 2)  # mm^2
    ma = 5e8  # N-mm, applied moment
    beamtype = 'PR'  # options: 'PR' (pin roller), 'FFr' (fix free), 'FF' (fix fix), 'FR' (fix roller)
    pointload = 10000  # N, point load
    uniformload = 1  # N/mm, uniform load
    sustainedduration = 60  # months (>=60 months)
    
    # Maximum permissible deflection
    max_defl = maxdeflection(clearspan, membertype)  # mm
    print(f"Maximum permissible deflection for {membertype}: {max_defl:.2f} mm")
    
    # Gross moment of inertia calculation
    ig = momentofinertia(width, height)  # mm^4
    print(f"Gross moment of inertia (Ig): {ig:.2f} mm^4")
    
    # Modulus of rupture calculation
    fr = modulusofrupture(ig, distance, maximummoment)  # N/mm^2
    print(f"Modulus of rupture (fr): {fr:.2f} N/mm^2")
    
    # Cracking moment calculation using NSCP Eq. 424.2.3.5b
    mcr = crackingmoment(fr, ig, height)  # N-mm
    print(f"Cracking moment (Mcr): {mcr:.2f} N-mm")
    
    # Compute the modular ratio and store it in a variable called modularratio.
    modularratio = calculate_modularratio(Es, Ec)
    
    # Cracked moment of inertia calculation
    icr = crackedmomentofinertia(width, distance, modularratio, asbar, height)  # mm^4
    print(f"Cracked moment of inertia (Icr): {icr:.2f} mm^4")
    
    # Effective moment of inertia calculation using NSCP Eq. 424.2.3.5a
    ie = effectivemomentofinertia(mcr, ma, ig, icr)  # mm^4
    print(f"Effective moment of inertia (Ie): {ie:.2f} mm^4")
    
    # Load deflection (short-term) for a PR (pin roller) beam
    sd = shortdeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload)  # mm
    print(f"Short-term deflection: {sd:.2f} mm")
    
    # Time-dependent factor for sustained loads
    timefactor = timedependentfactor(sustainedduration)  # dimensionless
    print(f"Time-dependent factor: {timefactor:.2f}")
    
    # Time-dependent deflection (long-term) (creep and shrinkage effects)
    ld = longdeflection(sd, timefactor, 16, width, height)  # mm
    print(f"Long-term deflection: {ld:.2f} mm")
    
    # Final deflection
    fd = finaldeflection(max_defl, ld, sd)  # mm
    print(f"Final deflection: {fd:.2f} mm")


Maximum permissible deflection for roof-undamaged: 25.00 mm
Gross moment of inertia (Ig): 450000000.00 mm^4
Modulus of rupture (fr): 333.33 N/mm^2
Cracking moment (Mcr): 1000000000.00 N-mm
Cracked moment of inertia (Icr): 270238934.21 mm^4
Effective moment of inertia (Ie): 450000000.00 mm^4
Short-term deflection: 13.75 mm
Time-dependent factor: 2.00
Long-term deflection: 23.55 mm
Final deflection: 25.00 mm


In [ ]:
import math

def maxdeflection(clearspan, membertype):
    """
    NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.

    Parameters:
    clearspan: span length (mm)
    membertype:
        'flat-roofs'    = flat roof members due to maximum lr and r
        'floor'         = floor members due to l
        'roof-damaged'  = roof or floors that are likely to be damaged by large deflections
        'roof-undamaged'= roof or floors that are not likely to be damaged by large deflection
    """
    if membertype == 'flat-roofs':
        return clearspan / 180
    elif membertype == 'floor':
        return clearspan / 360
    elif membertype == 'roof-damaged':
        return clearspan / 480
    elif membertype == 'roof-undamaged':
        return clearspan / 240
    else:
        raise ValueError("Invalid member type. Must be flat-roofs, floor, roof-damaged, or roof-undamaged")

def momentofinertia(width, height):
    ig = width * math.pow(height, 3) / 12  # in mm^4
    return ig

def modulusofrupture(ig, distance, maximummoment): 
    '''
    '''

    sectionmodulus = ig / distance 
    fr = maximummoment / sectionmodulus
    return fr

def crackingmoment(fr, ig, height):
    """
    NSCP 2015 Eq. 424.2.3.5b. Cracking moment  

    Parameters:
    ig: gross moment of inertia (mm^4)
    fr: modulus of rupture (MPa)
    yt: distance from neutral axis to extreme fiber (mm)
    """
    yt = height / 2
    mcr = (fr * ig) / yt
    return mcr

def calculate_modularratio(Es, Ec):
    # Calculate the modular ratio and round up to the nearest whole number.
    ratio = math.ceil(Es / Ec)
    
    # Enforce the minimum value of 6.
    if ratio >= 6:
        modularratio = ratio
    else:
        modularratio = 6
    return modularratio

def crackedmomentofinertia(width, distance, modularratio, asbar, depth):
    icr = width * math.pow(distance, 3) / 3 + modularratio * asbar * math.pow((depth - distance), 2)
    return icr

def effectivemomentofinertia(mcr, ma, ig, icr):
    """
    NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia

    Parameters:
    mcr: cracking moment (n-mm)
    ma: applied moment (n-mm)
    ig: gross moment of inertia (mm^4)
    icr: cracked section moment of inertia (mm^4)
    """
    ratio = math.pow(mcr / ma, 3)
    ie = ratio * ig + (1 - ratio) * icr
    if ie > ig:
        ie = ig
    return ie

def shortdeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload):
    """
    Calculates load (short-term) deflection based on beam type.

    Parameters:
    beamtype:
        'PR'  = pin roller (simply supported)
        'FFr' = fix free (cantilever)
        'FF'  = fix fix (fixed-fixed)
        'FR'  = fix roller (fixed-propped)
    elasticity: modulus of elasticity (N/mm^2)
    pointload: point load (N)
    uniformload: uniform load (N/mm)
    """
    loaddeflectionvalue = 0
    if beamtype == 'PR':  # pin roller (simply supported)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (48 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (5 * uniformload * math.pow(clearspan, 4)) / (384 * elasticity * ig)
    elif beamtype == 'FFr':  # fix free (cantilever)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (3 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (8 * elasticity * ig)
    elif beamtype == 'FF':  # fix fix (fixed-fixed)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (192 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (384 * elasticity * ig)
    elif beamtype == 'FR':  # fix roller (fixed-propped)
        if pointload:
            loaddeflectionvalue += (pointload * math.pow(clearspan, 3)) / (48 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * math.pow(clearspan, 4)) / (185 * elasticity * ig)
    else:
        raise ValueError("Invalid beam type specified. Must be PR, FFr, FF, or FR.")
    return loaddeflectionvalue

def timedependentfactor(sustainedduration):
    """
    NSCP 2015 Table 424.2.4.1.3
    Time-dependent factor for sustained loads.

    Parameters:
    sustainedduration: duration of sustained load in months
                       Valid values: 3, 6, 12, or >=60.
    """
    if sustainedduration == 3:
        timefactor = 1.0
    elif sustainedduration == 6:
        timefactor = 1.2
    elif sustainedduration == 12:
        timefactor = 1.4
    elif sustainedduration >= 60:
        timefactor = 2.0
    else:
        raise ValueError("Invalid sustainedduration. Must be 3, 6, 12, or >=60")
    return timefactor

def longdeflection(shortdeflectionvalue, timefactor, compressionbar, width, depth):
    """
    NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
    Additional deflection due to creep and shrinkage is the product of the immediate deflection and a factor lambda_delta.

    Parameters:
    deflection: immediate deflection due to sustained load (mm)
    timefactor: time-dependent factor based on sustained load duration
    compressionbar: diameter of the compression bar (mm)
    """
    asprime = math.pi * math.pow(compressionbar / 2, 2)
    rhoprime = asprime / (width * depth)
    lambda_delta = timefactor / (1 + 50 * rhoprime)
    timedependentdeflectionvalue = shortdeflectionvalue * lambda_delta
    return timedependentdeflectionvalue

def finaldeflection(maxdeflection, longdeflectionvalue, shortdeflectionvalue):
    """
    Determines the governing deflection as the sum of the time-dependent and load deflections,
    but capped by the maximum permissible deflection.
    """
    totaldeflection = longdeflectionvalue + shortdeflectionvalue
    if totaldeflection <= maxdeflection:
        return totaldeflection
    else:
        return maxdeflection

if __name__ == '__main__':
    clearspan = 6000  # mm
    membertype = 'roof-undamaged'  # options: 'flat-roofs', 'floor', 'roof-damaged', 'roof-undamaged'
    width = 200   # mm
    height = 300  # mm
    distance = height / 2  # mm (distance from neutral axis to extreme fiber)
    maximummoment = 1e9  # n-mm
    fcprime = 21
    Es = 200000
    Ec = 4700 * math.sqrt(fcprime)
    elasticity = 10000  # N/mm^2, modulus of elasticity of Steel-Concrete
    barsize = 16  # mm
    asbar = math.pi * math.pow(barsize / 2, 2)  # mm^2
    ma = 5e8  # n-mm, applied moment
    beamtype = 'PR'  # options: 'PR' (pin roller), 'FFr' (fix free), 'FF' (fix fix), 'FR' (fix roller)
    pointload = 10000   # N, point load
    uniformload = 1  # N/mm, uniform load
    sustainedduration = 60  # options: 3,6,12,>=60 months

    # Maximum permissible deflection
    max_defl = maxdeflection(clearspan, membertype)
    print(f"Maximum permissible deflection for {membertype}: {max_defl:.2f}")

    # Gross moment of inertia calculation
    ig = momentofinertia(width, height)
    print(f"Gross moment of inertia (Ig): {ig:.2f}")

    # Modulus of rupture calculation
    fr = modulusofrupture(ig, distance, maximummoment)
    print(f"Modulus of rupture (fr): {fr:.2f}")

    # Cracking moment calculation using NSCP Eq. 424.2.3.5b
    mcr = crackingmoment(fr, ig, height)
    print(f"Cracking moment (Mcr): {mcr:.2f}")

    # Compute the modular ratio and store it in a variable called modularratio.
    modularratio = calculate_modularratio(Es, Ec)
    
    # Cracked moment of inertia calculation
    icr = crackedmomentofinertia(width, distance, modularratio, asbar, height)
    print(f"Cracked moment of inertia (Icr): {icr:.2f}")

    # Effective moment of inertia calculation using NSCP Eq. 424.2.3.5a
    ie = effectivemomentofinertia(mcr, ma, ig, icr)
    print(f"Effective moment of inertia (Ie): {ie:.2f}")

    # Load deflection (short-term) for a PR (pin roller) beam
    sd = shortdeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload)
    print(f"Short-term deflection: {sd:.2f}")

    # Time-dependent factor for sustained loads
    timefactor = timedependentfactor(sustainedduration)
    print(f"Time-dependent factor: {timefactor:.2f}")

    # Time-dependent deflection (long-term) (creep and shrinkage effects)
    ld = longdeflection(sd, timefactor, 16, width, height)
    print(f"Long-term deflection: {ld:.2f}")

    # Final deflection
    fd = finaldeflection(max_defl, ld, sd)
    print(f"Final deflection: {fd:.2f}")

Maximum permissible deflection for roof-undamaged: 25.00
Gross moment of inertia (Ig): 450000000.00
Modulus of rupture (fr): 333.33
Cracking moment (Mcr): 1000000000.00
Cracked moment of inertia (Icr): 270238934.21
Effective moment of inertia (Ie): 450000000.00
Short-term deflection: 13.75
Time-dependent factor: 2.00
Long-term deflection: 23.55
Final deflection: 25.00


In [ ]:
# import math

# def maxdeflection(clearspan, membertype):
#     """
#     NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.

#     Parameters:
#     clearspan: span length (mm)
#     membertype:
#         'flatroofs'    = flat roof members due to maximum lr and r
#         'floor'       = floor members due to l
#         'roofdamaged' = roof or floors that are likely to be damaged by large deflections
#         'roofundamaged'= roof or floors that are not likely to be damaged by large deflection
#     """
#     if membertype == 'flat-roofs':
#         return clearspan / 180
#     elif membertype == 'floor':
#         return clearspan / 360
#     elif membertype == 'roofdamaged':
#         return clearspan / 480
#     elif membertype == 'roofundamaged':
#         return clearspan / 240
#     else:
#         raise ValueError("Invalid member type. Must be flatroofs, floor, roofdamaged, or roofundamaged")

# def momentofinertia(width, height):
#     ig = width * math.pow(height, 3) / 12  # in mm^4
#     return ig

# def modulusofrupture(ig, distance, maximummoment):
#     sectionmodulus = ig / distance 
#     fr = maximummoment / sectionmodulus
#     return fr

# def crackingmoment(fr, ig, height):
#     """
#     NSCP 2015 Eq. 424.2.3.5b. Cracking moment  

#     Parameters:
#     ig: gross moment of inertia (mm^4)
#     fr: modulus of rupture (MPa)
#     yt: distance from neutral axis to extreme fiber (mm)
#     """
#     yt = height / 2
#     mcr = (fr * ig) / yt
#     return mcr

# def crackedmomentofinertia(width, distance, modularratio, asbar, depth):
#     icr = width * math.pow(distance, 3) / 3 + modularratio * asbar * math.pow((depth - distance), 2)
#     return icr

# def effectivemomentofinertia(mcr, ma, ig, icr):
#     """
#     NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia

#     Parameters:
#     mcr: cracking moment (n-mm)
#     ma: applied moment (n-mm)
#     ig: gross moment of inertia (mm^4)
#     icr: cracked section moment of inertia (mm^4)
#     """
#     ratio = math.pow(mcr / ma, 3)
#     ie = ratio * ig + (1 - ratio) * icr
#     if ie > ig:
#         ie = ig
#     return ie

# def loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload):
#     loaddeflectionvalue = 0
#     if beamtype == 'simply supported':
#         if pointload:
#             loaddeflectionvalue += (pointload * clearspan ** 3) / (48 * elasticity * ig)
#         if uniformload:
#             loaddeflectionvalue += (5 * uniformload * clearspan ** 4) / (384 * elasticity * ig)
#     elif beamtype == 'cantilever':
#         if pointload:
#             loaddeflectionvalue += (pointload * clearspan ** 3) / (3 * elasticity * ig)
#         if uniformload:
#             loaddeflectionvalue += (uniformload * clearspan ** 4) / (8 * elasticity * ig)
#     elif beamtype == 'fixed fixed':
#         if pointload:
#             loaddeflectionvalue += (pointload * clearspan ** 3) / (192 * elasticity * ig)
#         if uniformload:
#             loaddeflectionvalue += (uniformload * clearspan ** 4) / (384 * elasticity * ig)
#     elif beamtype == 'fixed propped':
#         if pointload:
#             loaddeflectionvalue += (pointload * clearspan ** 3) / (48 * elasticity * ig)
#         if uniformload:
#             loaddeflectionvalue += (uniformload * clearspan ** 4) / (185 * elasticity * ig)
#     else:
#         raise ValueError("Invalid beam type specified.")
#     return loaddeflectionvalue

# def timedependentfactor(sustainedduration):
#     """
#     NSCP 2015 Table 424.2.4.1.3
#     Time-dependent factor for sustained loads.

#     Parameters:
#     sustainedduration: duration of sustained load in months
#                        Valid values: 3, 6, 12, or >=60.
#     """
#     if sustainedduration == 3:
#         timefactor = 1.0
#     elif sustainedduration == 6:
#         timefactor = 1.2
#     elif sustainedduration == 12:
#         timefactor = 1.4
#     elif sustainedduration >= 60:
#         timefactor = 2.0
#     else:
#         raise ValueError("Invalid sustainedduration. Must be 3, 6, 12, or >=60")
#     return timefactor

# def timedependentdeflection(deflection, timefactor, compressionbar, width, depth):
#     """
#     NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
#     Additional deflection due to creep and shrinkage is the product of the immediate deflection and a factor lambda_delta.

#     Parameters:
#     deflection: immediate deflection due to sustained load (mm)
#     timefactor: time-dependent factor based on sustained load duration
#     compressionbar: diameter of the compression bar (mm)
#     width: width of the section (mm)
#     depth: depth of the section (mm)
#     """
#     asprime = math.pi * math.pow(compressionbar / 2, 2)
#     rhoprime = asprime / (width * depth)
#     lambda_delta = timefactor / (1 + 50 * rhoprime)
#     timedependentdeflectionvalue = deflection * lambda_delta
#     return timedependentdeflectionvalue

# def finaldeflection(maxdeflection, timedependentdeflectionvalue, loaddeflectionvalue):
#     """
#     Determines the governing deflection as the sum of the time-dependent and load deflections,
#     but capped by the maximum permissible deflection.
#     """
#     totaldeflection = timedependentdeflectionvalue + loaddeflectionvalue
#     if totaldeflection <= maxdeflection:
#         return totaldeflection
#     else:
#         return maxdeflection

# if __name__ == '__main__':
#     clearspan = 6000  # mm
#     membertype = 'floor'  # options: 'flatroofs', 'floor', 'roofdamaged', 'roofundamaged'
#     width = 200   # mm
#     height = 300  # mm
#     distance = height / 2  # mm (distance from neutral axis to extreme fiber)
#     maximummoment = 1e9  # n-mm
#     modularratio = 10  # sample value
#     barsize = 16  # mm
#     asbar = math.pi * math.pow(barsize / 2, 2)  # mm^2
#     ma = 5e8  # n-mm, applied moment
#     beamtype = 'simply supported'
#     elasticity = 10000  # N/mm^2, sample modulus of elasticity
#     pointload = 10000   # N, sample point load
#     uniformload = 1  # N/mm, sample uniform load
#     sustainedduration = 60  # 3,6,12,>=60 months

#     # Maximum permissible deflection
#     max_defl = maxdeflection(clearspan, membertype)
#     print(f"Maximum permissible deflection for {membertype}: {max_defl:.2f}")

#     # Gross moment of inertia calculation
#     ig = momentofinertia(width, height)
#     print(f"Gross moment of inertia (Ig): {ig:.2f}")

#     # Modulus of rupture calculation
#     fr = modulusofrupture(ig, distance, maximummoment)
#     print(f"Modulus of rupture (fr): {fr:.2f}")

#     # Cracking moment calculation using NSCP Eq. 424.2.3.5b
#     mcr = crackingmoment(fr, ig, height)
#     print(f"Cracking moment (Mcr): {mcr:.2f}")

#     # Cracked moment of inertia calculation
#     icr = crackedmomentofinertia(width, distance, modularratio, asbar, height)
#     print(f"Cracked moment of inertia (Icr): {icr:.2f}")

#     # Effective moment of inertia calculation using NSCP Eq. 424.2.3.5a
#     ie = effectivemomentofinertia(mcr, ma, ig, icr)
#     print(f"Effective moment of inertia (Ie): {ie:.2f}")

#     # Load deflection for a simply supported beam
#     ld = loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload)
#     print(f"Load deflection: {ld:.2f}")

#     # Time-dependent factor for sustained loads
#     timefactor = timedependentfactor(sustainedduration)
#     print(f"Time-dependent factor: {timefactor:.2f}")

#     # Time-dependent deflection (creep and shrinkage effects)
#     tdd = timedependentdeflection(ld, timefactor, 16, width, height)
#     print(f"Time-dependent deflection: {tdd:.2f}")

#     # Final deflection
#     fd = finaldeflection(max_defl, tdd, ld)
#     print(f"Final deflection: {fd:.2f}")

Maximum permissible deflection for floor: 16.67
Gross moment of inertia (Ig): 450000000.00
Modulus of rupture (fr): 333.33
Cracking moment (Mcr): 1000000000.00
Cracked moment of inertia (Icr): 270238934.21
Effective moment of inertia (Ie): 450000000.00
Load deflection: 13.75
Time-dependent factor: 2.00
Time-dependent deflection: 23.55
Final deflection: 16.67
